In [1]:
import sys, os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

sys.path.append(os.path.join(os.getcwd(), '..'))
from src.util import *

In [2]:
with open('../data/results.pkl', 'rb') as f:
    results = pickle.load(f)

df = pd.read_csv('../data/test_cases.csv', usecols=range(0, 14))

In [3]:
df.head()
losses, rewards, mean_reward, fig1, fig2 = results[30]

In [4]:
mean_rewards = []

for index, row in df.iterrows():
    mean_reward = results[index][2]  
    mean_rewards.append(mean_reward)
df['mean_reward'] = mean_rewards

In [5]:
df['European_Price'] = df.apply(
    lambda row: black_scholes(
        S=row['s_0'],
        K=100, 
        T=row['t2'],
        r=row['r'],
        sigma=row['v'] if not np.isnan(row['v']) else row['sigma'],  # Use 'v' as sigma if not NaN, otherwise use 'sigma'
        option_type=row['h'].split('_')[-1].lower() 
    ),
    axis=1
)

In [6]:
call = lambda x, k: np.maximum(x-k, 0)
put  = lambda x, k: np.maximum(k-x, 0)

df['MC_Price'] = df.apply(
    lambda row: get_mc_price(
        generate_gbm_paths(
            nsim=1000,
            nstep=int(365 * row["t2"]),  
            t1=0,
            t2=row["t2"],
            s_0=row["s_0"],
            r=row["r"],
            q=row["q"],
            v=row["v"]
        ) if row["gbm"] else generate_heston_paths(
            nsim=1000,
            nstep=int(365 * row["t2"]),
            t1=0,
            t2=row["t2"],
            s_0=row["s_0"],
            r=row["r"],
            q=row["q"],
            v_0=row["v_0"],
            theta=row["theta"],
            rho=row["rho"],
            kappa=row["kappa"],
            sigma=row["sigma"]
        ),
        t1=0,
        t2=row["t2"],
        h=(call if row["h"].split('_')[-1].lower() == "call" else put), 
        k=100,
        r=row["r"]
    ),
    axis=1
)

In [7]:
df[df["case"] == "Base_Case"]


,case,s_0,v,t2,q,r,h,v_0,theta,rho,kappa,sigma,gbm,notes,mean_reward,European_Price,MC_Price
0,Base_Case,80,0.2,1.0,0.0,0.04,American_Put,NaN,NaN,NaN,NaN,NaN,True,"Base case with changing S_0, σ, and T for Amer...",19.932057,17.784517,18.203484
1,Base_Case,80,NaN,1.0,0.0,0.04,American_Put,0.04,0.04,-0.7,2.0,0.2,False,"Base case with changing S_0, σ, and T for Amer...",19.997333,17.784517,17.194915
2,Base_Case,80,0.2,2.0,0.0,0.04,American_Put,NaN,NaN,NaN,NaN,NaN,True,"Base case with changing S_0, σ, and T for Amer...",19.998591,17.049533,17.195765
3,Base_Case,80,NaN,2.0,0.0,0.04,American_Put,0.04,0.04,-0.7,2.0,0.2,False,"Base case with changing S_0, σ, and T for Amer...",19.977443,17.049533,16.290700
4,Base_Case,80,0.4,1.0,0.0,0.04,American_Put,NaN,NaN,NaN,NaN,NaN,True,"Base case with changing S_0, σ, and T for Amer...",20.023056,23.409238,23.732223
5,Base_Case,80,NaN,1.0,0.0,0.04,American_Put,0.16,0.16,-0.7,2.0,0.4,False,"Base case with changing S_0, σ, and T for Amer...",19.938423,23.409238,21.723078
6,Base_Case,80,0.4,2.0,0.0,0.04,American_Put,NaN,NaN,NaN,NaN,NaN,True,"Base case with changing S_0, σ, and T for Amer...",19.814753,25.936587,25.821546
7,Base_Case,80,NaN,2.0,0.0,0.04,American_Put,0.16,0.16,-0.7,2.0,0.4,False,"Base case with changing S_0, σ, and T for Amer...",20.038212,25.936587,25.907099
8,Base_Case,90,0.2,1.0,0.0,0.04,American_Put,NaN,NaN,NaN,NaN,NaN,True,"Base case with changing S_0, σ, and T for Amer...",9.956161,10.841383,11.414707
9,Base_Case,90,NaN,1.0,0.0,0.04,American_Put,0.04,0.04,-0.7,2.0,0.2,False,"Base case with changing S_0, σ, and T for Amer...",10.083580,10.841383,10.545634


In [8]:
fig1


<Figure size 2800x800 with 2 Axes>

In [9]:
fig2


<Figure size 2800x800 with 1 Axes>